In [1]:
import librosa
import numpy as np
import soundfile as sf
from pedalboard import Pedalboard, Phaser, Chorus, Reverb, load_plugin
import soundfile as sf
import scipy.signal as sps
from scipy.signal import butter, lfilter

# --- TOGGLE THESE FLAGS ---
DO_PITCH_SHIFT = False
PITCH_SHIFT_STEPS = -3  # e.g. -3 semitones to lower pitch

DO_RING_MOD = False
RING_MOD_FREQ = 400  # e.g. 30 Hz for metallic effect

DO_BANDPASS = True
BANDPASS_LOW = 300
BANDPASS_HIGH = 5000

DO_FLANGER = True            # Set to False to skip flanger
FLANGER_RATE = 1           # LFO rate in Hz, between 0.1 Hz and 5 Hz
FLANGER_DEPTH = 0.8          # Depth of modulation (0 to 1)
FLANGER_CENTRE_DELAY_MS = 3.0  # Centre delay in ms (lower values yield a flanger-like effect)
FLANGER_FEEDBACK = 0.5       # Amount of feedback (0 to ~0.9)
FLANGER_MIX = 0.2            # Wet/dry mix (0 = all dry, 1 = all wet)

DO_PHASER = True   # Toggle phaser
PHASER_RATE = 1.5 # Lower values (e.g., 0.1–1 Hz) yield a slow, sweeping modulation, which is typical for a classic phaser sound. 
                  # If you increase this value, the sweep will occur more quickly, which can create a more dramatic or choppy effect.
PHASER_DEPTH = 0.7 # Usually 0 to 1, where 0 means no modulation (effect is barely noticeable) and 1 means full modulation (the effect is very pronounced).
PHASER_MIX = 0.5 # Wet/dry mix (0 = all dry, 1 = all wet)

OUTPUT_FILE = 'corporate_voice.wav'  # Final exported filename


In [2]:
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    """
    Simple Butterworth bandpass filter to narrow frequency range
    (for an intercom / 'Evil Corp' effect).
    """
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return lfilter(b, a, data)

def apply_convolution_reverb(signal, ir):
    """
    Convolve the input signal with a room or speaker impulse response (IR).
    We'll trim it later to match the original length.
    """
    return sps.fftconvolve(signal, ir, mode='full')


In [ ]:
# Load the main audio file (MP3, WAV, OGG, FLAC, etc.)
# sr=None keeps original sample rate
y, sr = librosa.load('input.mp3', sr=None)

# We'll keep y_current as our "work-in-progress" audio.
y_current = y.copy()

print(f"Loaded {len(y)} samples at {sr} Hz.")


In [ ]:
if DO_PITCH_SHIFT:
    print(f"Pitch-shifting by {PITCH_SHIFT_STEPS} semitones...")
    y_current = librosa.effects.pitch_shift(
        y=y_current,
        sr=sr,
        n_steps=PITCH_SHIFT_STEPS,
        bins_per_octave=12,
        res_type='kaiser_best'
    )
else:
    print("Skipping pitch shift.")



In [ ]:
if DO_RING_MOD:
    print(f"Applying ring modulation at {RING_MOD_FREQ} Hz...")
    duration = len(y_current) / sr
    t = np.linspace(0, duration, len(y_current), endpoint=False)
    carrier = np.sin(2.0 * np.pi * RING_MOD_FREQ * t)
    y_current = y_current * carrier
else:
    print("Skipping ring modulation.")



In [ ]:
if DO_BANDPASS:
    print(f"Applying bandpass filter from {BANDPASS_LOW} Hz to {BANDPASS_HIGH} Hz...")
    y_current = bandpass_filter(y_current, BANDPASS_LOW, BANDPASS_HIGH, sr)
else:
    print("Skipping bandpass filter.")



In [ ]:
if DO_FLANGER:
    print(f"Applying Flanger: rate={FLANGER_RATE} Hz, depth={FLANGER_DEPTH}, centre_delay={FLANGER_CENTRE_DELAY_MS} ms, feedback={FLANGER_FEEDBACK}, mix={FLANGER_MIX}")
    
    # Convert mono audio to 2D array (channels x samples) for Pedalboard
    y_2d = np.expand_dims(y_current, axis=0)
    
    # Create a Chorus plugin instance tuned to simulate a flanger effect.
    # Note: A lower centre_delay and higher feedback help achieve a flanger-like sweep.
    from pedalboard import Pedalboard, Chorus
    flanger_plugin = Chorus(
        rate_hz=FLANGER_RATE,
        depth=FLANGER_DEPTH,
        centre_delay_ms=FLANGER_CENTRE_DELAY_MS,
        feedback=FLANGER_FEEDBACK,
        mix=FLANGER_MIX
    )
    
    # Build a Pedalboard chain with just the flanger (Chorus) effect
    board = Pedalboard([flanger_plugin])
    
    # Process the audio through the Pedalboard
    processed = board(y_2d, sample_rate=sr)
    
    # Convert back to a 1D array (mono)
    y_current = np.squeeze(processed, axis=0)
    print("Flanger applied.")
else:
    print("Skipping flanger.")



In [ ]:
if DO_PHASER:
    print(f"Applying Phaser with rate={PHASER_RATE} Hz, depth={PHASER_DEPTH}, mix={PHASER_MIX}...")
    y_2d = np.expand_dims(y_current, axis=0)

    board = Pedalboard([
        Phaser(rate_hz=PHASER_RATE, depth=PHASER_DEPTH, mix=PHASER_MIX),
    ])

    processed = board(y_2d, sample_rate=sr)
    y_current = np.squeeze(processed, axis=0)
else:
    print("Skipping phaser.")



In [ ]:
sf.write(OUTPUT_FILE, y_current, sr)
print(f"Done! Wrote processed audio to {OUTPUT_FILE}.")

